## rPlace Analysis
by: Daniel, Ryan, and Toby

In [2]:
# imports
import pandas as pd
import polars as pl
import gzip
import shutil
import os
import pyarrow.csv as csv
import pyarrow.parquet as pq
import pyarrow as pa
from pyarrow import parquet
import re
from datetime import datetime
import matplotlib.pyplot as plt

#### 1. Unzipping files stored in ZipFiles/ and storing them in UnzippedFiles/

In [ ]:
# This script will unzip all the files within the ZipFiles folder
source_directory = "ZipFiles"
directory_to_extract_to = "UnzippedFiles/"

# Loop over all files in the source directory
for filename in os.listdir(source_directory):
    if filename.endswith(".gzip"):
        path_to_gzip_file = os.path.join(source_directory, filename)

        # Remove "".gzip" and everything before number 
        # from the filename for the output file
        output_filename = filename[-11:-5] 
        output_file_path = os.path.join(directory_to_extract_to, output_filename)

        with gzip.open(path_to_gzip_file, "rb") as gzip_file:
            with open(output_file_path, "wb") as output_file:
                shutil.copyfileobj(gzip_file, output_file)
        print(f"Decompressed {filename}")

#### 2. Combining into parquet (Toby's version)

In [ ]:
def split_coords(coord_str):
    geometry = None
    #split it up
    coords = coord_str.split(',')
    if len(coords) == 4:
        #rectangle
        geometry = 'rect'
    elif len(coords) == 3:
        #circle
        geometry = 'circle'
    x = int(re.search(r'-?\d+', coords[0]).group())
    y = int(re.search(r'-?\d+', coords[1]).group())
    return (x,y,geometry)
	
source_directory = "UnzippedFiles/"
output_parquet_file = "rplace_new.parquet"

# Define schema for parquet
schema = pa.schema([
    pa.field("timestamp", pa.timestamp(unit='ms')),
    pa.field("user", pa.string()),
    #pa.field("coordinate", pa.string()),
    pa.field("pixel_color", pa.string()),
    pa.field("x", pa.int16()),
    pa.field("y", pa.int16()),
    pa.field("geometry", pa.string())
])

# Initialize parquet writer
writer = parquet.ParquetWriter(output_parquet_file, schema, compression="snappy")

for filename in os.listdir(source_directory):

    # Make sure file being processed is a csv
    if filename.endswith(".csv"):

        # Join source directory with current filename
        path_to_file = os.path.join(source_directory, filename)

        # Read current csv        
        print("Reading: " + filename)
        table = csv.read_csv(path_to_file)
        
        #Ensuring the date is in the proper format, requires removing UTC and redefining as timestamp in milliseconds
        array = table['timestamp'].combine_chunks()
        # Replace string ending with UTC
        array = pa.compute.replace_substring_regex(array, " UTC", "")
        # Convert to date
        array = array.cast(pa.timestamp(unit='ms'))
        # Put back in the table
        table = table.set_column(table.schema.get_field_index("timestamp"), "timestamp", array)
        
        #Create x,y,geometry and drop the coordinate column afterwards
        #This takes the coordinates, makes a series, then applies a custom function to split into x,y coords and the type of geometry if any
        coords = table['coordinate'].combine_chunks().to_pandas().apply(split_coords)
        coords = pd.DataFrame(coords.tolist(), columns=['x','y','geometry'])

        #add these columns in
        table = table.append_column("x", [coords['x']]) #stupid that you have to put these in a 2d array
        table = table.append_column("y", [coords['y']])
        table = table.append_column("geometry", [coords['geometry']])
        #drop the coordinate column now
        table = table.drop(['coordinate'])
        
        table = table.cast(schema)

        # Write current csv to parquet
        print("Writing: " + filename)
        writer.write_table(table)

# Close parquet writer if still open
if writer:
    writer.close()


print("Successfully combined into " + output_parquet_file)

#### 3. Aggregating data and outputting key files for further analysis

In [ ]:
# Scan in newly created parquet file
df = pl.scan_parquet("rplace_new.parquet")

In [ ]:
# Group by x and y coordinates and count occurrences
grouped_df = df.group_by(["x", "y"]).agg(pl.count("user").alias("count")).sort("count", descending=True)

# Display the result
top_pixels = grouped_df.head(100).collect()

# Write out to csv
top_pixels.write_csv("Highest used pixels.csv",separator=",")

In [ ]:
grouped_df = df.group_by(["user"]).agg(pl.count("x").alias("count")).sort("count", descending=True)

# Display the result
top_users = grouped_df.head(1000).collect()

top_users.write_csv("Highest active users.csv",separator=",")

In [ ]:
grouped_df = df.group_by(["pixel_color"]).agg(pl.count("x").alias("count")).sort("count", descending=True)

# Display the result
top_colors = grouped_df.head(100).collect()

top_colors.write_csv("Highest used colors.csv",separator=",")

#### 4. Visualization development and statistics

In [ ]:
# Average interval between pixel placement
grouped_df = df.filter(pl.col('user') == top_users['user'][2])
user_activity = grouped_df.collect().to_pandas()
user_activity_sort = user_activity.sort_values(by=['timestamp'],axis=0,ascending=True)
user_activity_sort['timestamp diff'] = user_activity_sort['timestamp'].diff().fillna(pd.Timedelta(seconds=0))

In [ ]:
user_activity_sort.sort_values(by='timestamp diff',ascending=False)[0:15]

In [ ]:
print(user_activity_sort['timestamp diff'].mean())
print(user_activity_sort['timestamp diff'].median())

#### 5. Shotbow Bots

In [ ]:
t = df.filter((pl.col('timestamp') >= datetime(2023,7,25,18,14)) & (pl.col('timestamp') <= datetime(2023,7,25,18,15,50)))
t.group_by(["user"]).agg(pl.count("x").alias("count")).sort("count", descending=True).collect()

In [ ]:
tt = t.collect()
tt.filter((pl.col('timestamp') >= datetime(2023,7,25,18,15,10)) & (pl.col('timestamp') <= datetime(2023,7,25,18,15,11)))

In [ ]:
ttt = tt.to_pandas()
ttt.plot.scatter(x='x', y='y', s=2, c='blue', alpha=0.05)  # c is color, alpha is transparency

plt.title('Pixel Positions on Shotbow Image Timeframe')
plt.xlabel('x')
plt.ylabel('y')
plt.grid(True)
plt.show()

In [ ]:
a = t.filter((pl.col('x') > 500) & (pl.col('x') < 1075) & (pl.col('y') < -125) & (pl.col('y') > -300))

In [ ]:
aa = a.group_by(["user"]).agg(pl.count("x").alias("count")).sort("count", descending=True)
aa.collect()

#### 6. Visualizations

In [ ]:
from plotnine import *

colordf = pd.read_csv('Highest used colors.csv')  

colordf['pixel_color'] = pd.Categorical(colordf['pixel_color'], categories=colordf['pixel_color'], ordered=True)
# Create the plot using ggplot
plot = (ggplot(colordf, aes(x='pixel_color', y='count', fill='pixel_color')) +
        geom_bar(stat='identity') +  # Create bars with heights corresponding to count values
        scale_fill_identity() +      # Use colors from the 'color' column
        labs(x='pixel_color', y='Count', title='Sorted Colors') +  # Labels and title
        theme_gray() +            # Minimalistic theme
        theme(axis_text_x=element_blank(),legend_position='none'))  # Hide legend

# Display the plot
print(plot)

In [ ]:
# Load data
userdf = pd.read_csv('Highest active users.csv')

# Convert 'user' column to categorical variable with order based on 'count'
userdf['user'] = pd.Categorical(userdf['user'], categories=userdf['user'], ordered=True)

# Create the plot using ggplot
plot = (ggplot(userdf[0:25], aes(x='user', y='count')) +
        geom_bar(stat='identity',fill='#FF5700') +  # Create bars with heights corresponding to count values
        labs(x='User', y='Count', title='Highest Active Users') +  # Labels and title
        theme_minimal() +            # Minimalistic theme
        theme(axis_text_x=element_blank(),  # Rotate x-axis labels for better readability
              legend_position='none'))  # Hide legend

# Display the plot
print(plot)